___
# Ciência dos Dados - PROJETO 2

___
## André Neustein

## Carla Gaieski

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [244]:
import pandas as pd
import os

In [245]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Carla Kim\Desktop\CD\DP\Projeto2-DPCD-master


In [246]:
import math
import matplotlib.pyplot as plt
import json
from random import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

In [247]:
#importando o arquivo
xlsx = pd.read_excel('spamhamV2.xlsx',sep=',')

In [248]:
#separando aleatoriamente 25% em teste e 75% em treinamento
train, test = train_test_split(xlsx, test_size = 0.25)
train.reset_index()
test.reset_index()

,index,Email,Class
0,1642,U WILL SWITCH YOUR FONE ON DAMMIT!!,ham
1,2210,Just gettin a bit arty with my collages at the...,ham
2,5258,1 in cbe. 2 in chennai.,ham
3,261,Anyway i'm going shopping on my own now. Cos m...,ham
4,4553,Try to do something dear. You read something f...,ham
5,4434,Don't b floppy... b snappy & happy! Only gay c...,spam
6,612,I have many dependents,ham
7,3534,I'm at home. Please call,ham
8,589,"I'm in a meeting, call me later at",ham
9,4495,"In case you wake up wondering where I am, I fo...",ham


In [249]:
#criando função para limpar os emails
def clean(email):
    email_final = email.replace("."," ").replace(","," ").replace("-"," ")\
    .replace(":"," ").replace(";"," ").replace("…"," ").replace("+"," ").replace('('," ").replace("'"," ").replace('"'," ")\
    .replace(')'," ").replace("@"," ").replace("#"," ").replace('  ',' ').replace('*',' ')
    return email_final

In [250]:
#limpando os emails contidos em treinamento

def limpar(mailtreino):
    trei = mailtreino
    return clean(trei)

LT = train.Email.apply(limpar,1)
train["Email"] = LT
train

C:\Users\Carla Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Email,Class
5099,Ah well that confuses things doesnt it? I thou...,ham
5499,Now get step 2 outta the way Congrats again,ham
2236,Give her something to drink if she takes it an...,ham
2488,I have 2 docs appointments next week / I m tir...,ham
563,GENT! We are trying to contact you Last weeken...,spam
2440,Back in brum! Thanks for putting us up and kee...,ham
1248,HI HUN! IM NOT COMIN 2NITE TELL EVERY1 IM SORR...,ham
5511,It‰Û÷s reassuring in this crazy world,ham
3575,The sign of maturity is not when we start sayi...,ham
2673,I am 6 ft We will be a good combination!,ham


___
## 3. Classificador Naive-Bayes

In [251]:
#criando dataframe de spams
trainspam = train.loc[train.Class == 'spam']
trainspam.head(2)

,Email,Class
563,GENT! We are trying to contact you Last weeken...,spam
3527,You are a å£1000 winner or Guaranteed Caller P...,spam


In [252]:
#criando dataframe de hams
trainham = train.loc[train.Class == 'ham']
trainham.head(2)

,Email,Class
5099,Ah well that confuses things doesnt it? I thou...,ham
5499,Now get step 2 outta the way Congrats again,ham


E-mails HAM:

In [253]:
#soma de tabela de emails ham
soma_ham = pd.Series(np.sum(trainham['Email']+'').split())

In [254]:
#fazendo uma lista de palavras que aparecem nos emails ham
palavras_ham = np.sum(trainham['Email']+'').split()

In [255]:
#calculo do total dos hams
total_ham = len(palavras_ham)

E-mails SPAM:

In [256]:
#soma de tabela spam
soma_spam = pd.Series(np.sum(trainspam['Email']+'').split())

In [257]:
#fazendo uma lista de palavras que aparecem nos spams
palavras_spam = np.sum(trainspam['Email']+'').split()


In [258]:
#calculo do total de spam
total_spam = len(palavras_spam)

Criando o classificador:

In [259]:
def classificador(mail):
    mail = mail.split(' ')
    prob_spam = [] #probabilidade de o email ser spam em train
    prob_ham = [] #probabilidade de o email ser ham em train
    prob_spam.append(train.Class.value_counts()['spam']/len(train))
    prob_ham.append(train.Class.value_counts()['ham']/len(train))
    for i in range(len(mail)):
        prob_spam.append((palavras_spam.count(mail[i])+1)/(total_spam+len(mail)))
        prob_ham.append((palavras_ham.count(mail[i])+1)/(total_ham+len(mail)))
    
    #multiplica os fatores da lista
    SPAM = np.prod(prob_spam)
    HAM = np.prod(prob_ham)
    
    if SPAM > HAM:
        return 'spam'
    else:
        return 'ham'
    

    Com as linhas de "E-mails SPAM" e "E-mails HAM" e a função classificador, foi possível estabelecer o classificador Naive Bayes do programa.
    As linhas de comando anteriores ao "Classificador" 

Testando nosso classificador para nossa base de dados test:

In [260]:
#limpando o dados de teste
def limpando(mailtest):
    walker = mailtest
    return clean(walker)

L = test.Email.apply(limpando,1)
test["Email"] = L

C:\Users\Carla Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [261]:
#função criada para testar nosso classificador
def testando(emailteste):
    oracao = emailteste
    return classificador(oracao)

T = test.Email.apply(testando,1)
test["Resultados"] = T

C:\Users\Carla Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [262]:
#visualizando o resultado do classificador, comparando-o com o dataframe original
test

,Email,Class,Resultados
1642,U WILL SWITCH YOUR FONE ON DAMMIT!!,ham,spam
2210,Just gettin a bit arty with my collages at the...,ham,ham
5258,1 in cbe 2 in chennai,ham,ham
261,Anyway i m going shopping on my own now Cos my...,ham,ham
4553,Try to do something dear You read something fo...,ham,ham
4434,Don t b floppy b snappy & happy! Only gay cha...,spam,spam
612,I have many dependents,ham,ham
3534,I m at home Please call,ham,ham
589,I m in a meeting call me later at,ham,ham
4495,In case you wake up wondering where I am I for...,ham,ham


    Nesta etapa, a função criada para testar o classificador Naive Bayes criado criou uma coluna em nosso data frame, denominada "Resultados", que mostram como o código classificou cada tipo de mensagem, de acordo com o treinamento que ele realizou anteriormete. Assim, realizando uma breve vizualização é possível perceber que houve acertos e alguns erros. Para que esses dados de acertos ou não, também conhecidos como falsos positivos,FP, (mensagens marcadas como SPAM mas que não são SPAM); positivos verdadeiros, PV, (mensagens marcadas como SPAM e são SPAM); falsos negativos, FN, (mensagens marcadas como não SPAM mas são SPAM) e negativos verdadeiros, NV, (mensagens marcadas como não SPAM e não são SPAM).

Descobrindo os valores de falsos positivos, positivos verdadeiros, falsos negativos e negativos verdadeiros:

In [263]:
#crosstab para vizualização da tablefa de distribuição conjunta referente à FP, PV, FN e NV
ct = pd.crosstab(test.Class,test.Resultados)
ct

Resultados,ham,spam
Class,,
ham,1087,120
spam,4,182


Positivos falsos:

In [264]:
FP = ct.ham['spam']/(ct.ham['spam']+ct.ham['ham'])*100
FP

0.36663611365719523

Negativos falsos:

Positivos verdadeiros:

Negativos verdadeiros: 

In [265]:
# descricao = []

# for i in range(len(test)):
#     if test.Class[i] == 'ham':
#         if test.Resultados[i] == 'ham':#ham e ham = positivo verdadeiro
#             descricao.append('PV')
            
#         if test.Resultados[i] == 'spam':#ham e spam = negativo falso
#             descricao.append('NF')
            
#     elif test.Class[i] == 'spam':
#         if test.Resultados[i] == 'spam':#spam e spam = negativo verdadeiro
#             descricao.append('NV')
            
#         if test.Resultados[i] == 'ham':#spam e ham = positivo falso
#             descricao.append('PF')

# corretos = pd.Series(descricao)

# pf = p.count('PF')/2
# pv = p.count('PV')/2
# nf = p.count('NF')/2
# nv = p.count('NV')/2

# print('Porcentagem de Positivos Verdadeiros: {}'.format(VB/2))
# print('Porcentagem de Negativos Falsos: {}'.format(FB/2))
# print('Porcentagens de Negativos Verdadeiros: {}'.format(VN/2))
# print('Porcentagem de Positivos falsos: {}'.format(FN/2))


___
## 4. Curva ROC

In [269]:
# df = pd.DataFrame(test)

In [270]:
def classificadoroc(mail):
    mail = mail.split(' ')
    prob_spam = []
    prob_ham = []
    prob_spam.append(train.Class.value_counts()['spam']/len(train))
    prob_ham.append(train.Class.value_counts()['ham']/len(train))
    for i in range(len(mail)):
        prob_spam.append((palavras_spam.count(mail[i])+1)/(total_spam+len(mail)))
        prob_ham.append((palavras_ham.count(mail[i])+1)/(total_ham+len(mail)))
    
    #multiplica os fatores da lista
    SPAM = np.prod(prob_spam)
    HAM = np.prod(prob_ham)
    
    if (SPAM) > n*(SPAM+HAM):
        return 'spam'
    else:
        return 'ham'

In [271]:
def rocando(emailrock):
    periodo = emailrock
    return classificadoroc(periodo)

# n=0.1
# P = test.apply(rocando,1)
# test['BayRoc'] = P
# test.head(2)

In [ ]:
ly = []
lx = []
coef = []

#for i in np.arange(0,1,0.01):
i=1
if i == 1:
    P = test.Email.apply(rocando,1)
    test["Roc"] = P
    ct = pd.crosstab(test.Class,test.Resultados)
    if hasattr (ct,'ham') and hasattr(ct,'spam'):
        y = ct.spam[1]/(ct.spam[1]+ct.spam[0])
        x = 1-(ct.ham[0]/(ct.ham[1]+ct.ham[0]))
        phi1 = (ct.spam["spam"]*ct.ham["ham"])-(ct.spam["ham"]*ct.ham["spam"])
        phi2 = (ct.spam["spam"]+ct.ham["spam"])*(ct.spam["spam"]+ct.spam["ham"])*(ct.ham["ham"]+ct.ham["spam"])*(ct.ham["ham"]+ct.spam["ham"])
        lx.append(x)
        ly.append(y)
        coef.append(phi1/(phi2**(1/2)))
    elif hasattr (ct,'ham') and not hasattr(ct,'spam'):
        x = 1
        y = 0 
        phi = 0
        lx.append(1-x)
        ly.append(y)
        coef.append(phi)
    elif hasattr (ct,'ham') and not hasattr(ct,'spam'):
        x = 0
        y = 1 
        phi = 0
        lx.append(1-x)
        ly.append(y)
        coef.append(phi)
        

In [ ]:
plt.plot(lx,ly)
plt.ylabel('Sensibilidade')
plt.xlabel('Especificidade')
plt.title('Curva ROC')
plt.show()

In [ ]:
coef

___
## 5. Qualidade do Classificador alterando a base de treinamento

In [ ]:
SS = []
HH = []

for i in range(1000):
    train, test = train_test_split(xlsx, test_size = 0.25)
    train.reset_index()
    test.reset_index()
    
    C = test.Email.apply(testando,1)
    test["Resultados"] = C
    
    ct3 = pd.crosstab(test.Class, test.Resultados)
    xx = ct.spam['spam']/(ct.spam['spam']+ct.spam['ham'])
    yy = ct.ham['ham']/(ct.ham['ham']+ct.ham['spam'])
    SS.append(xx)
    HH.append(yy)